In [1]:
import sys
sys.path.append("../../../")
import helper
from pandas import read_excel,DataFrame,melt,merge,read_csv,Series
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

### 행정동 코드 불러오기

In [4]:
origin=read_excel('./데이터파일/0.행정.xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN


### 지하철역 불러오기

In [5]:
subway=read_csv('./데이터파일/1.서울교통공사_지하철역_주소_및_전화번호_정보(완료).csv',encoding='euc-kr')
subway

,호선,역명,구주소,도로명주소,주소2,읍면동명
0,1호선,서울,서울특별시 중구 남대문로5가 73-6 서울역(1호선),서울특별시 중구 세종대로 지하2(남대문로 5가),중구 세종대로 지하2,회현동
1,1호선,시청,서울특별시 중구 정동 5-5 시청역(1호선),서울특별시 중구 세종대로 지하101(정동),중구 세종대로 지하101,소공동
2,1호선,종각,서울특별시 종로구 종로1가 54 종각역(1호선),서울특별시 종로구 종로 지하55(종로1가),종로구 종로 지하55,종로1.2.3.4가동
3,1호선,종로3가,서울특별시 종로구 종로3가 10-5 종로3가역(1호선),서울특별시 종로구 종로 지하129(종로3가),종로구 종로 지하129,종로1.2.3.4가동
4,1호선,종로5가,서울특별시 종로구 종로5가 82-1 종로5가역(1호선),서울특별시 종로구 종로 지하216(종로5가),종로구 종로 지하216,종로5.6가동
...,...,...,...,...,...,...
311,분당,선정릉(한국과학창의재단),서울특별시 강남구 삼성동 111-44,서울특별시 강남구 선릉로 580,강남구 선릉로 580,삼성제2동
312,분당,수서,서울특별시 강남구 수서동 728,서울특별시 강남구 광평로 270,강남구 광평로 270,수서동
313,분당,압구정로데오,서울특별시 강남구 압구정동 495,서울특별시 강남구 압구정로 402,강남구 압구정로 402,청담동
314,분당,왕십리,서울특별시 성동구 행당동 168-151,서울특별시 성동구 왕십리광장로 17,성동구 왕십리광장로 17,행당제1동


In [6]:
subway1=subway['읍면동명'].value_counts()
subway1=DataFrame(subway1)
subway1

,count
읍면동명,
한강로동,7
종로1.2.3.4가동,5
삼성제2동,5
상봉제1동,5
서교동,4
...,...
이화동,1
명동,1
방배제2동,1


In [7]:
subway1.reset_index('읍면동명',inplace=True)
subway2=subway1.rename(columns={'count':'지하철역_수'})
subway2

,읍면동명,지하철역_수
0,한강로동,7
1,종로1.2.3.4가동,5
2,삼성제2동,5
3,상봉제1동,5
4,서교동,4
...,...,...
199,이화동,1
200,명동,1
201,방배제2동,1
202,방화제3동,1


In [8]:
subway2['지하철역_수'].sum()

316

### 병원 불러오기

In [9]:
hospital=read_excel('./데이터파일/2.서울시_행정동별_의료기관_수(완료).xlsx')
hospital.head()

,동주소,의료기관_수
0,사직동,75
1,삼청동,11
2,부암동,4
3,평창동,15
4,무악동,6


In [10]:
hospital1=hospital.rename(columns={'동주소':'읍면동명'})
hospital1

,읍면동명,의료기관_수
0,사직동,75
1,삼청동,11
2,부암동,4
3,평창동,15
4,무악동,6
...,...,...
420,둔촌제1동,1
421,둔촌제2동,28
422,암사제1동,37
423,천호제2동,118


In [11]:
hospital1['의료기관_수'].sum()

17142

### 사회복지시설 불러오기

In [12]:
disables=read_excel('./데이터파일/3.서울시_사회복지시설_행정동.xlsx')
disables.head()

,시설명,시설코드,시설종류명(시설유형),시설종류상세명(시설종류),자치구(시)구분,시군구코드,시군구명,시설주소,구_주소,읍면동명
0,서울꽃동네신내노인요양원,A0001,(노인) 노인요양시설,노인의료복지시설,자치구,1126000000,중랑구,서울특별시 중랑구 신내로 194,중랑구,신내제1동
1,청운양로원,A0002,(노인) 양로시설,노인주거복지시설,자치구,1111000000,종로구,서울특별시 종로구 비봉길 76 (구기동),종로구,평창동
2,청운노인요양원,A0003,(노인) 노인요양시설,노인의료복지시설,자치구,1111000000,종로구,서울특별시 종로구 비봉길 76 (구기동),종로구,평창동
3,홍파양로원,A0004,(노인) 양로시설,노인주거복지시설,자치구,1135000000,노원구,서울특별시 노원구 동일로248길 30 (상계동),노원구,상계제1동
4,천사노인요양원,A0007,(노인) 노인요양시설,노인의료복지시설,자치구,1150000000,강서구,서울특별시 강서구 강서로45다길 30-22,강서구,화곡제3동


In [13]:
disables2=disables['읍면동명'].value_counts()
disables3=DataFrame(disables2)
disables3

,count
읍면동명,
구산동,27
등촌제3동,25
상계제1동,22
독산제1동,20
제기동,20
...,...
신수동,1
정릉제1동,1
소공동,1


In [14]:
disables3.reset_index('읍면동명',inplace=True)
disables4=disables3.rename(columns={'count':'사회복지시설_수'})
disables4

,읍면동명,사회복지시설_수
0,구산동,27
1,등촌제3동,25
2,상계제1동,22
3,독산제1동,20
4,제기동,20
...,...,...
397,신수동,1
398,정릉제1동,1
399,소공동,1
400,방배본동,1


In [15]:
disables4['사회복지시설_수'].sum()

2475

### 행정동별 인구수 불러오기(거주인구)

In [16]:
people=read_excel('./데이터파일/4.주민등록인구_2023년2월4일_기준(행정동)(완료).xlsx')
people.head()

,동별,세대 (세대),계(명),한국인 (명),등록외국인 (명),인구밀도
0,사직동,4570,9362,9089,273,2
1,삼청동,1127,2576,2277,299,2
2,부암동,4128,9377,9112,265,2
3,평창동,7209,17658,17476,182,2
4,무악동,3010,8015,7984,31,3


In [17]:
people1=people.rename(columns={'동별':'읍면동명'})
people1

,읍면동명,세대 (세대),계(명),한국인 (명),등록외국인 (명),인구밀도
0,사직동,4570,9362,9089,273,2
1,삼청동,1127,2576,2277,299,2
2,부암동,4128,9377,9112,265,2
3,평창동,7209,17658,17476,182,2
4,무악동,3010,8015,7984,31,3
...,...,...,...,...,...,...
421,둔촌제2동,10329,24169,24058,111,2
422,길동,21961,45142,44808,334,2
423,강일동,13109,32996,32811,185,3
424,상일1동,13618,39407,39283,124,3


In [18]:
people2=people1.filter(['읍면동명','계(명)'])
people2

,읍면동명,계(명)
0,사직동,9362
1,삼청동,2576
2,부암동,9377
3,평창동,17658
4,무악동,8015
...,...,...
421,둔촌제2동,24169
422,길동,45142
423,강일동,32996
424,상일1동,39407


### 행정동별 면적 및 비율 불러오기

In [19]:
area=read_excel('./데이터파일/5.행정구역_면적및비율(동별)(완료).xlsx')
area.head()

,동별,면적 (k㎡),구성비 (%),행정,법정,소계,소계.1
0,종로구소계,23.91,3.95,17,87,279,1475
1,사직동,1.23,0.20,1,12,16,68
2,삼청동,1.49,0.25,1,7,10,42
3,부암동,2.27,0.38,1,3,18,98
4,평창동,8.87,1.47,1,2,19,147


#### 각 자치구별 소계 행 제거

In [20]:
area1 = area[~area['동별'].str.contains('소계')] #소계라는 단어가 들어가 있는 행만 걸러서 drop
area1

,동별,면적 (k㎡),구성비 (%),행정,법정,소계,소계.1
1,사직동,1.23,0.20,1,12,16,68
2,삼청동,1.49,0.25,1,7,10,42
3,부암동,2.27,0.38,1,3,18,98
4,평창동,8.87,1.47,1,2,19,147
5,무악동,0.36,0.06,1,1,12,71
...,...,...,...,...,...,...,...
446,둔촌제1동,0.92,0.15,1,1,23,229
447,둔촌제2동,1.56,0.26,1,-,29,260
448,강일동,2.26,0.37,1,1,38,296
449,상일1동,1.82,0.30,1,1,46,397


In [21]:
area2=area1.filter(['동별','면적 (k㎡)','구성비 (%)'])
area2

,동별,면적 (k㎡),구성비 (%)
1,사직동,1.23,0.20
2,삼청동,1.49,0.25
3,부암동,2.27,0.38
4,평창동,8.87,1.47
5,무악동,0.36,0.06
...,...,...,...
446,둔촌제1동,0.92,0.15
447,둔촌제2동,1.56,0.26
448,강일동,2.26,0.37
449,상일1동,1.82,0.30


In [22]:
area3=area2.rename(columns={'동별':'읍면동명'})
area3

,읍면동명,면적 (k㎡),구성비 (%)
1,사직동,1.23,0.20
2,삼청동,1.49,0.25
3,부암동,2.27,0.38
4,평창동,8.87,1.47
5,무악동,0.36,0.06
...,...,...,...
446,둔촌제1동,0.92,0.15
447,둔촌제2동,1.56,0.26
448,강일동,2.26,0.37
449,상일1동,1.82,0.30


### 공원 불러오기

In [23]:
park=read_csv('./데이터파일/6.서울시_주요_공원현황(완료).csv',encoding='euc-kr')
park.head()

,연번,공원명,면적,개원일,주요시설,주요식물,지역,공원주소,주소,읍면동명,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84)
0,1,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,중구,서울특별시 중구 삼일대로 231(예장동),중구 삼일대로 231,필동,198364.107,450395.554,126.990377,37.550140
1,2,길동생태공원,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),강동구 천호대로 1291,길동,213554.120,448852.675,127.154779,37.540394
2,4,서울숲,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),성동구 뚝섬로 273,성수1가제1동,203695.432,449290.726,127.041798,37.543072
3,5,월드컵공원,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),마포구 하늘공원로 84,상암동,190658.070,451598.831,126.878907,37.571805
4,6,광화문시민열린마당,8948.1㎡,1999.1.1,조 경 시 설 : 육조상징벽 6개소 전통담장 4개소 열주 22조 화계6개소 편 익 ...,수 목 : 소나무 느티나무 배롱나무등 27종 5535주 초 화 : 맥문동 꽃창포 수...,종로구,서울특별시 종로구 세종로 76-2,종로구 세종로 76-2,청운효자동,198054.179,452832.049,126.978023,37.575045


In [24]:
park1=park['읍면동명'].value_counts()
park1

읍면동명
사직동            3
진관동            3
양재제2동          3
장지동            2
내곡동            2
              ..
종로1.2.3.4가동    1
일원본동           1
답십리제2동         1
대방동            1
세곡동            1
Name: count, Length: 109, dtype: int64

In [25]:
park2=DataFrame(park1)
park2

,count
읍면동명,
사직동,3
진관동,3
양재제2동,3
장지동,2
내곡동,2
...,...
종로1.2.3.4가동,1
일원본동,1
답십리제2동,1


In [26]:
park3=park2.reset_index('읍면동명')
park4=park3.rename(columns={'count':'행정동별_공원수'})
park4

,읍면동명,행정동별_공원수
0,사직동,3
1,진관동,3
2,양재제2동,3
3,장지동,2
4,내곡동,2
...,...,...
104,종로1.2.3.4가동,1
105,일원본동,1
106,답십리제2동,1
107,대방동,1


In [27]:
park4['행정동별_공원수'].sum()

129

### 체육시설 불러오기

In [28]:
gym=read_excel('./데이터파일/7.결과주소_전국공공체육시설_위경도포함_수정(완료).xlsx')
gym.head()

,FCLTY_NM,FCLTY_FLAG_NM,INDUTY_CD,INDUTY_NM,FCLTY_TY_CD,POSESN_MBY_CTPRVN_NM,POSESN_MBY_SIGNGU_CD,POSESN_MBY_SIGNGU_NM,RSPNSBLTY_DEPT_NM,RSPNSBLTY_TEL_NO,...,ROAD_NM_CTPRVN_NM,ROAD_NM_EMD_CD,ROAD_NM_EMD_NM,RDNMADR_NM,FCLTY_LO,FCLTY_LA,_PARCEL_AD,_ROAD_AD,구_주소,동_주소
0,돌 뫼,공공,P08,간이운동장,P0801,서울특별시,1129000000,성북구,NaN,NaN,...,서울특별시,NaN,NaN,NaN,127.052763,37.605125,서울특별시 성북구 석관동 산 1-77,서울특별시 성북구 화랑로18길 92,성북구,석관동
1,동 방,공공,P08,간이운동장,P0801,서울특별시,1129000000,성북구,NaN,NaN,...,서울특별시,NaN,NaN,NaN,127.043551,37.616625,서울특별시 성북구 장위동 219-223,서울특별시 성북구 장위로15길 66,성북구,장위제1동
2,동 산,공공,P08,간이운동장,P0801,서울특별시,1129000000,성북구,NaN,NaN,...,서울특별시,NaN,NaN,NaN,127.032290,37.605577,서울특별시 성북구 하월곡동 90-166,서울특별시 성북구 종암로36길 14,성북구,월곡제1동
3,밤 골,공공,P08,간이운동장,P0801,서울특별시,1129000000,성북구,NaN,NaN,...,서울특별시,NaN,NaN,NaN,127.039928,37.607571,서울특별시 성북구 하월곡동 산 2-140,서울특별시 성북구 화랑로13가길 17-8,성북구,월곡제1동
4,삼 선,공공,P08,간이운동장,P0801,서울특별시,1129000000,성북구,NaN,NaN,...,서울특별시,NaN,NaN,NaN,127.008989,37.582602,서울특별시 성북구 삼선동1가 303,서울특별시 성북구 삼선동1가 삼선교로4길 133,성북구,삼선동


In [29]:
gym2=gym.filter(['FCLTY_NM','INDUTY_NM','_PARCEL_AD','구_주소','동_주소'])
gym2

,FCLTY_NM,INDUTY_NM,_PARCEL_AD,구_주소,동_주소
0,돌 뫼,간이운동장,서울특별시 성북구 석관동 산 1-77,성북구,석관동
1,동 방,간이운동장,서울특별시 성북구 장위동 219-223,성북구,장위제1동
2,동 산,간이운동장,서울특별시 성북구 하월곡동 90-166,성북구,월곡제1동
3,밤 골,간이운동장,서울특별시 성북구 하월곡동 산 2-140,성북구,월곡제1동
4,삼 선,간이운동장,서울특별시 성북구 삼선동1가 303,성북구,삼선동
...,...,...,...,...,...
1279,방배e편한세상,간이운동장,서울특별시 서초구 방배동 884-21,서초구,방배제1동
1280,현대ESA,간이운동장,서울특별시 서초구 서초동 1338,서초구,서초제2동
1281,ESA2차,간이운동장,서울특별시 서초구 서초동 1542-6,서초구,서초제3동
1282,KT앞 둔치,간이운동장,서울특별시 서초구 우면동 143,서초구,양재제1동


In [30]:
gym3=gym2['동_주소'].value_counts()
gym4=DataFrame(gym3)
gym4.head()

,count
동_주소,
혜화동,39
양재제1동,22
신도림동,19
공릉제2동,19
양재제2동,18


In [31]:
gym4.reset_index('동_주소',inplace=True)
gym5=gym4.rename(columns={'동_주소':'읍면동명','count':'공공체육시설_수'})
gym5

,읍면동명,공공체육시설_수
0,혜화동,39
1,양재제1동,22
2,신도림동,19
3,공릉제2동,19
4,양재제2동,18
...,...,...
290,답십리2동,1
291,독산제2동,1
292,대조동,1
293,신내제2동,1


In [32]:
gym5['공공체육시설_수'].sum()

1284

### 문화시설 불러오기

In [33]:
culture=read_excel('./데이터파일/8.서울시_문화시설_현황_(한국어)_행정동주소(완료).xlsx')
culture.head()

,순번,분류명,공간명(한글),개관년도,지역,주소,구_주소,동_주소
0,1,전시시설,[의외의 조합]_청년예술공간지원사업 선정공간,2016,중구,서울시 중구 동호로17길 121 (신당동),중구,다산동
1,2,전시시설,[17717]_청년예술공간지원사업 선정공간,2014,성북구,서울시 성북구 성북로8길 11 (성북동),성북구,성북동
2,3,전시시설,[space55]_청년예술공간지원사업 선정공간,2017,은평구,서울시 은평구 증산로19길 9-3 (신사동),은평구,신사제1동
3,4,공연장,[게토얼라이브]_청년예술공간지원사업 선정공간,2016,성동구,"서울시 성동구 왕십리로 104 (성수동1가, 영화빌딩)",성동구,성수1가제2동
4,5,전시시설,[대안예술공간이포]_청년예술공간지원사업 선정공간,2008,영등포구,서울시 영등포구 도림로126길 9 (문래동3가),영등포구,문래동


In [34]:
culture1=culture.filter(['분류명','공간명(한글)','주소','동_주소'])
culture1

,분류명,공간명(한글),주소,동_주소
0,전시시설,[의외의 조합]_청년예술공간지원사업 선정공간,서울시 중구 동호로17길 121 (신당동),다산동
1,전시시설,[17717]_청년예술공간지원사업 선정공간,서울시 성북구 성북로8길 11 (성북동),성북동
2,전시시설,[space55]_청년예술공간지원사업 선정공간,서울시 은평구 증산로19길 9-3 (신사동),신사제1동
3,공연장,[게토얼라이브]_청년예술공간지원사업 선정공간,"서울시 성동구 왕십리로 104 (성수동1가, 영화빌딩)",성수1가제2동
4,전시시설,[대안예술공간이포]_청년예술공간지원사업 선정공간,서울시 영등포구 도림로126길 9 (문래동3가),문래동
...,...,...,...,...
792,다문화_가족센터,용산구 가족센터,서울시 용산구 이태원로 224-19 3층,한남동
793,다문화_가족센터,은평구 가족센터,서울시 은평구 은평로21가길 15-17,녹번동
794,다문화_가족센터,종로구 가족센터,서울시 종로구 종로53길 29(창신동 197-17) 토월 2층,창신제1동
795,다문화_가족센터,서울중구 가족센터,서울시 중구 퇴계로 460 중구종합복지센터 10층,신당5동


In [35]:
culture2=culture1.rename(columns={'동_주소':'읍면동명'})
culture3=culture2.filter(['공간명(한글)','읍면동명'])
culture3

,공간명(한글),읍면동명
0,[의외의 조합]_청년예술공간지원사업 선정공간,다산동
1,[17717]_청년예술공간지원사업 선정공간,성북동
2,[space55]_청년예술공간지원사업 선정공간,신사제1동
3,[게토얼라이브]_청년예술공간지원사업 선정공간,성수1가제2동
4,[대안예술공간이포]_청년예술공간지원사업 선정공간,문래동
...,...,...
792,용산구 가족센터,한남동
793,은평구 가족센터,녹번동
794,종로구 가족센터,창신제1동
795,서울중구 가족센터,신당5동


In [36]:
culture4=culture3['읍면동명'].value_counts()
culture5=DataFrame(culture4)


In [37]:
culture6=culture5.rename(columns={'count':'문화시설_수'})

In [38]:
culture7=culture6.reset_index('읍면동명')
culture7['문화시설_수'].sum()

797

In [39]:
culture7

,읍면동명,문화시설_수
0,이화동,108
1,혜화동,64
2,종로1.2.3.4가동,40
3,삼청동,31
4,청담동,27
...,...,...
239,천호제3동,1
240,방화제2동,1
241,수유제3동,1
242,반포제3동,1


### 학교 불러오기

In [40]:
school=read_excel('./데이터파일/9.학교_수정_행정동추가(완료).xlsx')
school.head()

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,남녀공학구분명,고등학교구분명,산업체특별학급존재여부,고등학교일반실업구분명,설립일자,개교기념일,시도교육청코드,적재일시,구_주소,동_주소
0,각종학교(중),사립,7134155,선화예술중학교,Sunhwa Arts Middle School,서울특별시성동광진교육지원청,4991,서울특별시 광진구 천호대로 664,/ 선화예술중고등학교 (능동),02-2204-1100,남여공학,NaN,N,일반계,19731201,19730705,B10,20230627,광진구,능동
1,초등학교,공립,7134150,서울숭신초등학교,Seoul Soongshin Elementary School,서울특별시성동광진교육지원청,4702,서울특별시 성동구 마장로 161,(하왕십리동),02-2252-5950,남여공학,NaN,N,일반계,19590403,19590403,B10,20230615,성동구,왕십리도선동
2,중학교,공립,7134142,행당중학교,Haengdang Middle School,서울특별시성동광진교육지원청,4764,서울특별시 성동구 왕십리로 189,(행당동/행당중학교),02-2292-2721,남여공학,NaN,N,일반계,19680806,19681002,B10,20230615,성동구,사근동
3,중학교,사립,7134141,한양대학교사범대학부속중학교,Hanyang University Middle School,서울특별시성동광진교육지원청,4761,서울특별시 성동구 마조로 42,(사근동/한양사대부속중?고등학교),02-2200-3700,남여공학,NaN,N,일반계,19600118,19600118,B10,20230615,성동구,사근동
4,중학교,공립,7134140,자양중학교,Jayang Middle School,서울특별시성동광진교육지원청,5069,서울특별시 광진구 뚝섬로41길 33,(자양동/서울자양중학교),02-446-0365,남여공학,NaN,N,일반계,19840301,19840301,B10,20230615,광진구,자양제3동


In [41]:
school1=school.filter(['학교종류명','학교명','구_주소','동_주소'])
school1.head()

,학교종류명,학교명,구_주소,동_주소
0,각종학교(중),선화예술중학교,광진구,능동
1,초등학교,서울숭신초등학교,성동구,왕십리도선동
2,중학교,행당중학교,성동구,사근동
3,중학교,한양대학교사범대학부속중학교,성동구,사근동
4,중학교,자양중학교,광진구,자양제3동


In [42]:
school2=school1['동_주소'].value_counts()
school3=DataFrame(school2)
school3.head()

,count
동_주소,
발산제1동,15
신정제3동,13
개포제2동,13
대방동,13
하계제1동,12


In [43]:
school3.reset_index('동_주소',inplace=True)
school4=school3.rename(columns={'동_주소':'읍면동명','count':'학교_수'})
school4

,읍면동명,학교_수
0,발산제1동,15
1,신정제3동,13
2,개포제2동,13
3,대방동,13
4,하계제1동,12
...,...,...
386,마천제1동,1
387,마천제2동,1
388,명일제1동,1
389,문정제2동,1


### 사업체 불러오기

In [44]:
company=read_excel('./데이터파일/10.서울시_사업체_2020(완료).xlsx')
company.head()

,Unnamed: 0,기준연도,행정구역분류코드,사업체구분코드,산업세세분류,자영업자_계,무급가족종사자_계,상용종사자,임시일용종사자_계,무급종사자_계,T종사자_합계,TM_X,TM_Y,행정구,읍면동명
0,0,2018,1122053,1,56113,1,0,0,4,0,5,200708.083119,543082.520931,서초구,서초제3동
1,1,2018,1111056,1,91139,1,0,0,0,0,1,206682.387429,558980.517849,노원구,공릉제2동
2,2,2018,1108077,1,47129,1,0,3,0,0,4,204542.497168,556801.964655,성북구,장위제2동
3,3,2018,1104068,1,45219,1,0,1,0,0,2,204753.544846,549837.996919,성동구,성수2가제3동
4,4,2018,1108082,1,56113,1,0,0,0,0,1,200772.838148,554315.709485,성북구,삼선동


In [45]:
company1=company['읍면동명'].value_counts()
company2=DataFrame(company1)
company2.head()

,count
읍면동명,
역삼제1동,15691
종로1.2.3.4가동,15501
가산동,15057
신당동,11544
서교동,9910


In [46]:
company2.reset_index('읍면동명',inplace=True)
company3=company2.rename(columns={'count':'사업체_수'})
company3.head()

,읍면동명,사업체_수
0,역삼제1동,15691
1,종로1.2.3.4가동,15501
2,가산동,15057
3,신당동,11544
4,서교동,9910


In [47]:
company3['사업체_수'].sum()

813131

### 주차장 불러오기

In [48]:
parkingarea=read_excel('./데이터파일/11.주차장(동별)(2022년).xlsx')
parkingarea.head()

,읍면동명,개소 (공영+민영),면수 (공영+민영)
0,종로구소계,8232,92062
1,사직동,362,12605
2,삼청동,343,2237
3,부암동,596,3533
4,평창동,1617,9390


In [49]:
# '동별' 컬럼에 '소계'를 포함하고 있는 행 삭제
parkingarea = parkingarea[~parkingarea['읍면동명'].str.contains('소계')]
parkingarea

,읍면동명,개소 (공영+민영),면수 (공영+민영)
1,사직동,362,12605
2,삼청동,343,2237
3,부암동,596,3533
4,평창동,1617,9390
5,무악동,69,3185
...,...,...,...
446,천호제2동,1673,16529
447,길동,1734,17661
448,강일동,103,16089
449,상일1동,501,19552


### 근처 수변 포함하는 행정동 불러오기

In [50]:
# QGIS에서 버퍼로 변경 800m설정 후 행정동 경계와 교집합하여 나온 행정동
# 이후 속성테이블에서 데이터 복사해서 엑셀에서 가공

river=read_excel('./데이터파일/0.행정.xlsx',sheet_name=2)
river

,하천 및 한강 반경 800m 포함 행정동,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,fid,BASE_DATE,ADM_CD,읍면동명,하천및한강여부
1,1,20220630,11010550,부암동,유
2,2,20220630,11010560,평창동,유
3,3,20220630,11010670,창신제1동,유
4,4,20220630,11010690,창신제3동,유
...,...,...,...,...,...
327,328,20220630,11250720,암사제1동,유
328,329,20220630,11250730,천호제2동,유
329,330,20220630,11250750,강일동,유
330,331,20220630,11250760,상일1동,유


In [51]:
river1=river.drop(0)
river1.head()

,하천 및 한강 반경 800m 포함 행정동,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
1,1,20220630,11010550,부암동,유
2,2,20220630,11010560,평창동,유
3,3,20220630,11010670,창신제1동,유
4,4,20220630,11010690,창신제3동,유
5,5,20220630,11010700,숭인제1동,유


In [52]:
river2=river1.rename(columns={'Unnamed: 3':'읍면동명','Unnamed: 4':'하천및한강여부'})
river2.head()

,하천 및 한강 반경 800m 포함 행정동,Unnamed: 1,Unnamed: 2,읍면동명,하천및한강여부
1,1,20220630,11010550,부암동,유
2,2,20220630,11010560,평창동,유
3,3,20220630,11010670,창신제1동,유
4,4,20220630,11010690,창신제3동,유
5,5,20220630,11010700,숭인제1동,유


In [53]:
river3=river2.filter(['읍면동명','하천및한강여부'])
river3

,읍면동명,하천및한강여부
1,부암동,유
2,평창동,유
3,창신제1동,유
4,창신제3동,유
5,숭인제1동,유
...,...,...
327,암사제1동,유
328,천호제2동,유
329,강일동,유
330,상일1동,유


### 아동 인구 불러오기

In [54]:
kids=read_excel('./데이터파일/12.주민등록인구(연령별_동별)_아동.xlsx')
kids.head()

,동별,0~4세,5~9세,합계(0~9세)
0,종로구소계,2556,3880,6436
1,사직동,178,310,488
2,삼청동,36,72,108
3,부암동,117,268,385
4,평창동,411,622,1033


In [55]:
# '동별' 컬럼에 '소계'단어를 포함하고 있는 행 삭제
kids = kids[~kids['동별'].str.contains('소계')]
kids

,동별,0~4세,5~9세,합계(0~9세)
1,사직동,178,310,488
2,삼청동,36,72,108
3,부암동,117,268,385
4,평창동,411,622,1033
5,무악동,207,355,562
...,...,...,...,...
446,암사제1동,755,1019,1774
447,천호제2동,749,769,1518
448,길동,931,1130,2061
449,상일1동,1578,3022,4600


In [56]:
kids1=kids.filter(['동별','합계(0~9세)'])
kids1.head()

,동별,합계(0~9세)
1,사직동,488
2,삼청동,108
3,부암동,385
4,평창동,1033
5,무악동,562


In [57]:
kids1.rename(columns={'동별':'읍면동명','합계(0~9세)':'아동인구(0~9세)'},inplace=True)
print(kids1.head())
kids1['아동인구(0~9세)'].sum()

  읍면동명  아동인구(0~9세)
1  사직동         488
2  삼청동         108
3  부암동         385
4  평창동        1033
5  무악동         562


531432

In [58]:
kids1

,읍면동명,아동인구(0~9세)
1,사직동,488
2,삼청동,108
3,부암동,385
4,평창동,1033
5,무악동,562
...,...,...
446,암사제1동,1774
447,천호제2동,1518
448,길동,2061
449,상일1동,4600


### 노인 인구(65세)이상 불러오기

In [59]:
old=read_excel('./데이터파일/13.고령자현황_수정됨.xlsx')
old

,읍면동별,소계
0,종로구소계,27958
1,사직동,1808
2,삼청동,626
3,부암동,1814
4,평창동,3495
...,...,...
446,암사제1동,6303
447,천호제2동,6140
448,길동,8478
449,상일1동,4839


In [60]:
# '읍면동별' 컬럼에 '소계'단어를 포함하고 있는 행 삭제
old = old[~old['읍면동별'].str.contains('소계')]
old

,읍면동별,소계
1,사직동,1808
2,삼청동,626
3,부암동,1814
4,평창동,3495
5,무악동,1476
...,...,...
446,암사제1동,6303
447,천호제2동,6140
448,길동,8478
449,상일1동,4839


In [61]:
old1=old.rename(columns={'읍면동별':'읍면동명','소계':'노인인구(65세이상)'})
old1.head()

,읍면동명,노인인구(65세이상)
1,사직동,1808
2,삼청동,626
3,부암동,1814
4,평창동,3495
5,무악동,1476


### 장애인 인구 불러오기

In [62]:
dis=read_excel('./데이터파일/14.장애인+현황(장애유형별_동별)_수정.xlsx')
dis

,읍면동별,계
0,종로구소계,5834
1,사직동,267
2,삼청동,110
3,부암동,311
4,평창동,491
...,...,...
446,천호제2동,1648
447,길동,1988
448,강일동,1661
449,상일1동,1084


In [63]:
# '읍면동별' 컬럼에 '소계'단어를 포함하고 있는 행 삭제
dis = dis[~dis['읍면동별'].str.contains('소계')]
dis

,읍면동별,계
1,사직동,267
2,삼청동,110
3,부암동,311
4,평창동,491
5,무악동,325
...,...,...
446,천호제2동,1648
447,길동,1988
448,강일동,1661
449,상일1동,1084


In [64]:
dis1=dis.rename(columns={'계':'장애인_인구'})
dis1

,읍면동별,장애인_인구
1,사직동,267
2,삼청동,110
3,부암동,311
4,평창동,491
5,무악동,325
...,...,...
446,천호제2동,1648
447,길동,1988
448,강일동,1661
449,상일1동,1084


In [65]:
dis1.rename(columns={'읍면동별':'읍면동명'},inplace=True)
dis1.head()

,읍면동명,장애인_인구
1,사직동,267
2,삼청동,110
3,부암동,311
4,평창동,491
5,무악동,325


### 도시공원 면적(행정동별 1세대당) 불러오기

In [66]:
ntparkarea=read_csv('./데이터파일/15.서울시_읍면동별_1세대당_도시공원_면적.csv',encoding='euc-kr')
ntparkarea

,행정구역ID,행정구역,세대당공원면적
0,1111051500,청운효자동,337.55
1,1111053000,사직동,60.79
2,1111054000,삼청동,956.51
3,1111055000,부암동,0.00
4,1111056000,평창동,1.14
...,...,...,...
420,1174065000,성내제2동,0.18
421,1174066000,성내제3동,0.35
422,1174068500,길동,4.11
423,1174069000,둔촌제1동,0.00


In [67]:
ntparkarea1=ntparkarea.filter(['행정구역','세대당공원면적'])
ntparkarea2=ntparkarea1.rename(columns={'행정구역':'읍면동명'})
ntparkarea2.head()

,읍면동명,세대당공원면적
0,청운효자동,337.55
1,사직동,60.79
2,삼청동,956.51
3,부암동,0.00
4,평창동,1.14


### 보육시설 불러오기

In [68]:
daycare=read_excel('./데이터파일/16.보육시설(동별).xlsx')
daycare.head()

,읍면동명,보육시설수
0,사직동,0
1,삼청동,3
2,부암동,3
3,평창동,3
4,무악동,3


### 전처리 완료된 파일들 순차적 병합 진행

origin, subway2, hospital1, disables4, people2, area3, park4, gym5, culture7, school4, company3, parkingarea, river3

kids1, old1, dis1, ntparkarea2

In [69]:
tmp=merge(origin,subway2,how='left')
tmp

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN
...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0


In [70]:
tmp2=merge(tmp,hospital1,how='left')
tmp2

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0
...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0


In [71]:
tmp3=merge(tmp2,disables4,how='left')
tmp3

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN


In [72]:
tmp4=merge(tmp3,people2,how='left')
tmp4

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수,계(명)
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0,11700
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0,9362
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN,2576
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0,9377
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0,17658
...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0,23303
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0,22882
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0,45142
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN,70


In [73]:
tmp5=merge(tmp4,area3,how='left')
tmp5

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수,계(명),면적 (k㎡),구성비 (%)
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0,11700,2.57,0.42
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0,9362,1.23,0.20
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN,2576,1.49,0.25
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0,9377,2.27,0.38
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0,17658,8.87,1.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0,23303,0.67,0.11
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0,22882,0.71,0.12
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0,45142,1.61,0.27
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN,70,0.92,0.15


In [74]:
tmp6=merge(tmp5,park4,how='left')
tmp6

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수,계(명),면적 (k㎡),구성비 (%),행정동별_공원수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0,11700,2.57,0.42,2.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0,9362,1.23,0.20,3.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN,2576,1.49,0.25,1.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0,9377,2.27,0.38,1.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0,17658,8.87,1.47,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0,23303,0.67,0.11,NaN
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0,22882,0.71,0.12,NaN
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0,45142,1.61,0.27,1.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN,70,0.92,0.15,NaN


In [75]:
tmp7=merge(tmp6,gym5,how='left')
tmp7

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수,계(명),면적 (k㎡),구성비 (%),행정동별_공원수,공공체육시설_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0,11700,2.57,0.42,2.0,10.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0,9362,1.23,0.20,3.0,8.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN,2576,1.49,0.25,1.0,2.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0,9377,2.27,0.38,1.0,5.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0,17658,8.87,1.47,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0,23303,0.67,0.11,NaN,NaN
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0,22882,0.71,0.12,NaN,NaN
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0,45142,1.61,0.27,1.0,1.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN,70,0.92,0.15,NaN,NaN


In [76]:
tmp8=merge(tmp7,culture7,how='left')
tmp8

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수,계(명),면적 (k㎡),구성비 (%),행정동별_공원수,공공체육시설_수,문화시설_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0,11700,2.57,0.42,2.0,10.0,6.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0,9362,1.23,0.20,3.0,8.0,13.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN,2576,1.49,0.25,1.0,2.0,31.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0,9377,2.27,0.38,1.0,5.0,5.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0,17658,8.87,1.47,NaN,5.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0,23303,0.67,0.11,NaN,NaN,2.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0,22882,0.71,0.12,NaN,NaN,1.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0,45142,1.61,0.27,1.0,1.0,1.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN,70,0.92,0.15,NaN,NaN,NaN


In [77]:
tmp9=merge(tmp8,school4,how='left')
tmp9

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수,계(명),면적 (k㎡),구성비 (%),행정동별_공원수,공공체육시설_수,문화시설_수,학교_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0,11700,2.57,0.42,2.0,10.0,6.0,8.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0,9362,1.23,0.20,3.0,8.0,13.0,2.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN,2576,1.49,0.25,1.0,2.0,31.0,2.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0,9377,2.27,0.38,1.0,5.0,5.0,4.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0,17658,8.87,1.47,NaN,5.0,11.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0,23303,0.67,0.11,NaN,NaN,2.0,NaN
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0,22882,0.71,0.12,NaN,NaN,1.0,NaN
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0,45142,1.61,0.27,1.0,1.0,1.0,4.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN,70,0.92,0.15,NaN,NaN,NaN,4.0


### 전처리 완료된 파일들 순차적 병합 진행

company3, parkingarea, river3, kids1, old1, dis1, ntparkarea2

In [78]:
tmp10=merge(tmp9,company3,how='left')
tmp10

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,사회복지시설_수,계(명),면적 (k㎡),구성비 (%),행정동별_공원수,공공체육시설_수,문화시설_수,학교_수,사업체_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,5.0,11700,2.57,0.42,2.0,10.0,6.0,8.0,992.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,1.0,9362,1.23,0.20,3.0,8.0,13.0,2.0,3532.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,NaN,2576,1.49,0.25,1.0,2.0,31.0,2.0,708.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,4.0,9377,2.27,0.38,1.0,5.0,5.0,4.0,558.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,11.0,17658,8.87,1.47,NaN,5.0,11.0,4.0,747.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,6.0,23303,0.67,0.11,NaN,NaN,2.0,NaN,1889.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,6.0,22882,0.71,0.12,NaN,NaN,1.0,NaN,2451.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,11.0,45142,1.61,0.27,1.0,1.0,1.0,4.0,5153.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,NaN,70,0.92,0.15,NaN,NaN,NaN,4.0,15.0


In [79]:
tmp11=merge(tmp10,parkingarea,how='left')
tmp11.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,계(명),면적 (k㎡),구성비 (%),행정동별_공원수,공공체육시설_수,문화시설_수,학교_수,사업체_수,개소 (공영+민영),면수 (공영+민영)
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,11700,2.57,0.42,2.0,10.0,6.0,8.0,992.0,668,4592
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,9362,1.23,0.20,3.0,8.0,13.0,2.0,3532.0,362,12605
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,2576,1.49,0.25,1.0,2.0,31.0,2.0,708.0,343,2237
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,9377,2.27,0.38,1.0,5.0,5.0,4.0,558.0,596,3533
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,17658,8.87,1.47,NaN,5.0,11.0,4.0,747.0,1617,9390


In [80]:
tmp12=merge(tmp11,river3,how='left')
tmp12.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,면적 (k㎡),구성비 (%),행정동별_공원수,공공체육시설_수,문화시설_수,학교_수,사업체_수,개소 (공영+민영),면수 (공영+민영),하천및한강여부
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,2.57,0.42,2.0,10.0,6.0,8.0,992.0,668,4592,NaN
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,1.23,0.20,3.0,8.0,13.0,2.0,3532.0,362,12605,NaN
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,1.49,0.25,1.0,2.0,31.0,2.0,708.0,343,2237,NaN
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,2.27,0.38,1.0,5.0,5.0,4.0,558.0,596,3533,유
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,8.87,1.47,NaN,5.0,11.0,4.0,747.0,1617,9390,유


In [81]:
tmp13=merge(tmp12,kids1,how='left')
tmp13.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,구성비 (%),행정동별_공원수,공공체육시설_수,문화시설_수,학교_수,사업체_수,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세)
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,0.42,2.0,10.0,6.0,8.0,992.0,668,4592,NaN,612.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,0.20,3.0,8.0,13.0,2.0,3532.0,362,12605,NaN,488.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,0.25,1.0,2.0,31.0,2.0,708.0,343,2237,NaN,108.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,0.38,1.0,5.0,5.0,4.0,558.0,596,3533,유,385.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,1.47,NaN,5.0,11.0,4.0,747.0,1617,9390,유,1033.0


In [82]:
tmp14=merge(tmp13,old1,how='left')
tmp14.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,행정동별_공원수,공공체육시설_수,문화시설_수,학교_수,사업체_수,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상)
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,2.0,10.0,6.0,8.0,992.0,668,4592,NaN,612.0,2154
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,3.0,8.0,13.0,2.0,3532.0,362,12605,NaN,488.0,1808
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,1.0,2.0,31.0,2.0,708.0,343,2237,NaN,108.0,626
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,1.0,5.0,5.0,4.0,558.0,596,3533,유,385.0,1814
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,NaN,5.0,11.0,4.0,747.0,1617,9390,유,1033.0,3495


In [83]:
tmp15=merge(tmp14,dis1,how='left')
tmp15.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,공공체육시설_수,문화시설_수,학교_수,사업체_수,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,10.0,6.0,8.0,992.0,668,4592,NaN,612.0,2154,437
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,8.0,13.0,2.0,3532.0,362,12605,NaN,488.0,1808,267
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,2.0,31.0,2.0,708.0,343,2237,NaN,108.0,626,110
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,5.0,5.0,4.0,558.0,596,3533,유,385.0,1814,311
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,5.0,11.0,4.0,747.0,1617,9390,유,1033.0,3495,491


In [84]:
tmp16=merge(tmp15,ntparkarea2,how='left')
tmp16.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,문화시설_수,학교_수,사업체_수,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,6.0,8.0,992.0,668,4592,NaN,612.0,2154,437,337.55
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,13.0,2.0,3532.0,362,12605,NaN,488.0,1808,267,60.79
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,31.0,2.0,708.0,343,2237,NaN,108.0,626,110,956.51
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,5.0,4.0,558.0,596,3533,유,385.0,1814,311,0.00
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,11.0,4.0,747.0,1617,9390,유,1033.0,3495,491,1.14


In [85]:
tmp17=merge(tmp16,daycare,how='left')
tmp17.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,학교_수,사업체_수,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,8.0,992.0,668,4592,NaN,612.0,2154,437,337.55,4.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,2.0,3532.0,362,12605,NaN,488.0,1808,267,60.79,0.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,2.0,708.0,343,2237,NaN,108.0,626,110,956.51,3.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,4.0,558.0,596,3533,유,385.0,1814,311,0.00,3.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,4.0,747.0,1617,9390,유,1033.0,3495,491,1.14,3.0


In [86]:
scaler=MinMaxScaler()

In [87]:
tmp17.isna().sum()

시도명              0
시군구코드            0
시군구명             0
행정동코드            0
행정동코드_수정         0
읍면동명             0
생성일자             0
말소일자           426
지하철역_수         222
의료기관_수           3
사회복지시설_수        29
계(명)             0
면적 (k㎡)          0
구성비 (%)          0
행정동별_공원수       317
공공체육시설_수       139
문화시설_수         182
학교_수            35
사업체_수            5
개소 (공영+민영)       0
면수 (공영+민영)       0
하천및한강여부         95
아동인구(0~9세)       2
노인인구(65세이상)      0
장애인_인구           0
세대당공원면적          3
보육시설수            1
dtype: int64

In [88]:
tmp18 = tmp17.copy()
tmp18['세대당공원면적minmax']=scaler.fit_transform(tmp18[['세대당공원면적']])
tmp18.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,사업체_수,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,992.0,668,4592,NaN,612.0,2154,437,337.55,4.0,0.352898
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,3532.0,362,12605,NaN,488.0,1808,267,60.79,0.0,0.063554
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,708.0,343,2237,NaN,108.0,626,110,956.51,3.0,1.000000
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,558.0,596,3533,유,385.0,1814,311,0.00,3.0,0.000000
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,747.0,1617,9390,유,1033.0,3495,491,1.14,3.0,0.001192


In [89]:
water_score=read_excel('수변_행정동별_점수화.xlsx')
water_score1=water_score.rename(columns={'행정동명':'읍면동명','count':'수변점수'})
water_score1

,읍면동명,수변점수
0,흑석동,10
1,신내제1동,10
2,수서동,10
3,수유제2동,10
4,수유제3동,10
...,...,...
323,논현제1동,1
324,월곡제1동,1
325,항동,1
326,창신3동,1


In [90]:
tmp19=tmp18.merge(water_score1,on='읍면동명',how='left')
tmp19

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,NaN,NaN,11.0,...,668,4592,NaN,612.0,2154,437,337.55,4.0,0.352898,NaN
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,NaN,2.0,75.0,...,362,12605,NaN,488.0,1808,267,60.79,0.0,0.063554,NaN
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,NaN,NaN,11.0,...,343,2237,NaN,108.0,626,110,956.51,3.0,1.000000,NaN
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,NaN,NaN,4.0,...,596,3533,유,385.0,1814,311,0.00,3.0,0.000000,10.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,NaN,NaN,15.0,...,1617,9390,유,1033.0,3495,491,1.14,3.0,0.001192,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,NaN,3.0,62.0,...,1278,7869,유,897.0,4469,1075,0.18,9.0,0.000188,4.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,NaN,1.0,80.0,...,1241,10656,유,1024.0,4098,913,0.35,11.0,0.000366,9.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,NaN,1.0,103.0,...,1734,17661,NaN,2061.0,8478,1988,4.11,17.0,0.004297,NaN
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,NaN,1.0,1.0,...,16,193,유,1.0,11,3,0.00,0.0,0.000000,7.0


In [91]:
tmp20=tmp19.fillna(0)
tmp20

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,개소 (공영+민영),면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0.0,0.0,11.0,...,668,4592,0,612.0,2154,437,337.55,4.0,0.352898,0.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0.0,2.0,75.0,...,362,12605,0,488.0,1808,267,60.79,0.0,0.063554,0.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0.0,0.0,11.0,...,343,2237,0,108.0,626,110,956.51,3.0,1.000000,0.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0.0,0.0,4.0,...,596,3533,유,385.0,1814,311,0.00,3.0,0.000000,10.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0.0,0.0,15.0,...,1617,9390,유,1033.0,3495,491,1.14,3.0,0.001192,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0.0,3.0,62.0,...,1278,7869,유,897.0,4469,1075,0.18,9.0,0.000188,4.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0.0,1.0,80.0,...,1241,10656,유,1024.0,4098,913,0.35,11.0,0.000366,9.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0.0,1.0,103.0,...,1734,17661,0,2061.0,8478,1988,4.11,17.0,0.004297,0.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0.0,1.0,1.0,...,16,193,유,1.0,11,3,0.00,0.0,0.000000,7.0


In [92]:
bus_stop=read_excel('./데이터파일/17.서울시_버스정류소_위치정보_행정동주소.xlsx')
bus_stop.head()

,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입,동_주소
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로,종로1.2.3.4가동
1,100000002,1002,창경궁.서울대학교병원,126.996522,37.579433,중앙차로,종로1.2.3.4가동
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로,혜화동
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로,종로1.2.3.4가동
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로,혜화동


In [93]:
bus_stop1=bus_stop['동_주소'].value_counts()
bus_stop2=DataFrame(bus_stop1)
bus_stop2.head()

,count
동_주소,
가산동,104
상암동,96
서교동,80
서초제3동,75
양재제1동,73


In [94]:
bus_stop2.reset_index('동_주소',inplace=True)
bus_stop3=bus_stop2.rename(columns={'동_주소':'읍면동명','count':'행정동별버스정류장수'})
bus_stop3.head()

,읍면동명,행정동별버스정류장수
0,가산동,104
1,상암동,96
2,서교동,80
3,서초제3동,75
4,양재제1동,73


In [95]:
bus_stop3['읍면동명'].value_counts()

읍면동명
가산동        1
수서동        1
이태원제2동     1
논현제1동      1
중곡제1동      1
          ..
안암동        1
상계제6.7동    1
상계제1동      1
항동         1
장암동        1
Name: count, Length: 428, dtype: int64

In [96]:
tmp21=tmp20.merge(bus_stop3,on='읍면동명',how='left')
tmp21

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0.0,0.0,11.0,...,4592,0,612.0,2154,437,337.55,4.0,0.352898,0.0,32
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0.0,2.0,75.0,...,12605,0,488.0,1808,267,60.79,0.0,0.063554,0.0,27
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0.0,0.0,11.0,...,2237,0,108.0,626,110,956.51,3.0,1.000000,0.0,16
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0.0,0.0,4.0,...,3533,유,385.0,1814,311,0.00,3.0,0.000000,10.0,34
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0.0,0.0,15.0,...,9390,유,1033.0,3495,491,1.14,3.0,0.001192,10.0,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0.0,3.0,62.0,...,7869,유,897.0,4469,1075,0.18,9.0,0.000188,4.0,18
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0.0,1.0,80.0,...,10656,유,1024.0,4098,913,0.35,11.0,0.000366,9.0,15
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0.0,1.0,103.0,...,17661,0,2061.0,8478,1988,4.11,17.0,0.004297,0.0,36
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0.0,1.0,1.0,...,193,유,1.0,11,3,0.00,0.0,0.000000,7.0,9


In [97]:
tmp21.isna().sum()

시도명              0
시군구코드            0
시군구명             0
행정동코드            0
행정동코드_수정         0
읍면동명             0
생성일자             0
말소일자             0
지하철역_수           0
의료기관_수           0
사회복지시설_수         0
계(명)             0
면적 (k㎡)          0
구성비 (%)          0
행정동별_공원수         0
공공체육시설_수         0
문화시설_수           0
학교_수             0
사업체_수            0
개소 (공영+민영)       0
면수 (공영+민영)       0
하천및한강여부          0
아동인구(0~9세)       0
노인인구(65세이상)      0
장애인_인구           0
세대당공원면적          0
보육시설수            0
세대당공원면적minmax    0
수변점수             0
행정동별버스정류장수       0
dtype: int64

In [98]:
tmp22=tmp21.fillna(0)
tmp22.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,면수 (공영+민영),하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0.0,0.0,11.0,...,4592,0,612.0,2154,437,337.55,4.0,0.352898,0.0,32
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0.0,2.0,75.0,...,12605,0,488.0,1808,267,60.79,0.0,0.063554,0.0,27
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0.0,0.0,11.0,...,2237,0,108.0,626,110,956.51,3.0,1.000000,0.0,16
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0.0,0.0,4.0,...,3533,유,385.0,1814,311,0.00,3.0,0.000000,10.0,34
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0.0,0.0,15.0,...,9390,유,1033.0,3495,491,1.14,3.0,0.001192,10.0,54


### 유통업체 불러오기

In [99]:
leisure=read_excel('./데이터파일/18.유통업체현황_.xlsx')
leisure.head()

,읍면동명,합계개소,대형마트개소,백화점개소,전문점개소,쇼핑센터개소,복합쇼핑개소,기타대규모점포개소
0,종로구소계,13.0,-,-,2,-,-,11
1,사직동,NaN,-,-,-,-,-,-
2,삼청동,NaN,-,-,-,-,-,-
3,부암동,NaN,-,-,-,-,-,-
4,평창동,NaN,-,-,-,-,-,-


In [100]:
leisure1=leisure.filter(['읍면동명','합계개소'])
leisure1.head()

,읍면동명,합계개소
0,종로구소계,13.0
1,사직동,NaN
2,삼청동,NaN
3,부암동,NaN
4,평창동,NaN


In [101]:
leisure2=leisure1.rename(columns={'합계개소':'행정동별유통업체수'})
leisure2.head()

,읍면동명,행정동별유통업체수
0,종로구소계,13.0
1,사직동,NaN
2,삼청동,NaN
3,부암동,NaN
4,평창동,NaN


In [102]:
# '동별' 컬럼에 '소계'단어를 포함하고 있는 행 삭제
leisure2 = leisure2[~leisure2['읍면동명'].str.contains('소계')]
leisure2

,읍면동명,행정동별유통업체수
1,사직동,NaN
2,삼청동,NaN
3,부암동,NaN
4,평창동,NaN
5,무악동,NaN
...,...,...
446,천호제2동,3.0
447,길동,2.0
448,강일동,NaN
449,상일1동,NaN


In [103]:
tmp23=merge(tmp22,leisure2,how='left')
tmp23

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,하천및한강여부,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0.0,0.0,11.0,...,0,612.0,2154,437,337.55,4.0,0.352898,0.0,32,NaN
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0.0,2.0,75.0,...,0,488.0,1808,267,60.79,0.0,0.063554,0.0,27,NaN
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0.0,0.0,11.0,...,0,108.0,626,110,956.51,3.0,1.000000,0.0,16,NaN
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0.0,0.0,4.0,...,유,385.0,1814,311,0.00,3.0,0.000000,10.0,34,NaN
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0.0,0.0,15.0,...,유,1033.0,3495,491,1.14,3.0,0.001192,10.0,54,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0.0,3.0,62.0,...,유,897.0,4469,1075,0.18,9.0,0.000188,4.0,18,1.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0.0,1.0,80.0,...,유,1024.0,4098,913,0.35,11.0,0.000366,9.0,15,1.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0.0,1.0,103.0,...,0,2061.0,8478,1988,4.11,17.0,0.004297,0.0,36,2.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0.0,1.0,1.0,...,유,1.0,11,3,0.00,0.0,0.000000,7.0,9,NaN


### 공공자전거 대여소 불러오기

In [104]:
publicbikestop=read_excel('./데이터파일/19.공공자전거_대여소_정보_행정동(좌표).xlsx')
publicbikestop.head()

,대여소\n번호,보관소(대여소)명,소재지(위치)-자치구,상세주소,위도,경도,설치\n시기,거치\n대수(LCD),거치\n대수(QR),운영\n방식,동_주소
0,301,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,2015-10-07,20.0,20.0,QR,사직동
1,302,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,2015-10-07,12.0,12.0,QR,사직동
2,303,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,2015-10-07,8.0,8.0,QR,사직동
3,305,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,2015-01-07,16.0,16.0,QR,종로1.2.3.4가동
4,307,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,2015-10-07,11.0,11.0,QR,사직동


In [105]:
publicbikestop1=publicbikestop['동_주소'].value_counts()
publicbikestop2=DataFrame(publicbikestop1)
publicbikestop2.head()

,count
동_주소,
여의동,46
가양제1동,44
상암동,32
진관동,30
종로1.2.3.4가동,28


In [106]:
publicbikestop2.reset_index('동_주소',inplace=True)
publicbikestop3=publicbikestop2.rename(columns={'동_주소':'읍면동명','count':'공공자전거대여소수'})
publicbikestop3

,읍면동명,공공자전거대여소수
0,여의동,46
1,가양제1동,44
2,상암동,32
3,진관동,30
4,종로1.2.3.4가동,28
...,...,...
413,정릉제1동,1
414,화곡제8동,1
415,돈암제2동,1
416,화곡제4동,1


In [107]:
tmp24=merge(tmp23,publicbikestop3,how='left')
tmp24

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0.0,0.0,11.0,...,612.0,2154,437,337.55,4.0,0.352898,0.0,32,NaN,3.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0.0,2.0,75.0,...,488.0,1808,267,60.79,0.0,0.063554,0.0,27,NaN,12.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0.0,0.0,11.0,...,108.0,626,110,956.51,3.0,1.000000,0.0,16,NaN,5.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0.0,0.0,4.0,...,385.0,1814,311,0.00,3.0,0.000000,10.0,34,NaN,5.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0.0,0.0,15.0,...,1033.0,3495,491,1.14,3.0,0.001192,10.0,54,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0.0,3.0,62.0,...,897.0,4469,1075,0.18,9.0,0.000188,4.0,18,1.0,5.0
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0.0,1.0,80.0,...,1024.0,4098,913,0.35,11.0,0.000366,9.0,15,1.0,6.0
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0.0,1.0,103.0,...,2061.0,8478,1988,4.11,17.0,0.004297,0.0,36,2.0,7.0
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0.0,1.0,1.0,...,1.0,11,3,0.00,0.0,0.000000,7.0,9,NaN,4.0


In [108]:
tmp24.isna().sum()
tmp25=tmp24.fillna(0)
tmp25.isna().sum()

시도명              0
시군구코드            0
시군구명             0
행정동코드            0
행정동코드_수정         0
읍면동명             0
생성일자             0
말소일자             0
지하철역_수           0
의료기관_수           0
사회복지시설_수         0
계(명)             0
면적 (k㎡)          0
구성비 (%)          0
행정동별_공원수         0
공공체육시설_수         0
문화시설_수           0
학교_수             0
사업체_수            0
개소 (공영+민영)       0
면수 (공영+민영)       0
하천및한강여부          0
아동인구(0~9세)       0
노인인구(65세이상)      0
장애인_인구           0
세대당공원면적          0
보육시설수            0
세대당공원면적minmax    0
수변점수             0
행정동별버스정류장수       0
행정동별유통업체수        0
공공자전거대여소수        0
dtype: int64

In [109]:
tmp25.to_excel('행정동병합테스트22.xlsx',index=False)

In [3]:
test11=read_excel('행정동병합테스트22.xlsx')
test11.head(2)

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,488,1808,267,60.79,0,0.063554,0,27,0,12


In [4]:
test11.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수'],
      dtype='object')

In [5]:
test11['인구밀도']=test11['계(명)']/test11['면적 (k㎡)']
test11.head(2)

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114


In [ ]:
test11

In [7]:
test11.to_excel('행정동병합테스트22.xlsx',index=False)

In [8]:
test12=read_excel('행정동병합테스트22.xlsx')
test12

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,62,...,4469,1075,0.18,9,0.000188,4,18,1,5,34780.597015
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,80,...,4098,913,0.35,11,0.000366,9,15,1,6,32228.169014
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,103,...,8478,1988,4.11,17,0.004297,0,36,2,7,28038.509317
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,1,...,11,3,0.00,0,0.000000,7,9,0,4,76.086957


In [10]:
to_remove=['내곡동', '양재제2동', '암사제2동', '암사제3동', '고덕제2동', '중곡제4동', '공릉제2동',
'상계제3.4동', '도봉제1동', '우이동', '인수동', '정릉제4동', '진관동', '방화제2동', '공항동', '항동',
'시흥제3동', '시흥제5동', '대학동', '남현동']
test13 = test12[~test12['읍면동명'].isin(to_remove)]
test13

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,62,...,4469,1075,0.18,9,0.000188,4,18,1,5,34780.597015
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,80,...,4098,913,0.35,11,0.000366,9,15,1,6,32228.169014
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,103,...,8478,1988,4.11,17,0.004297,0,36,2,7,28038.509317
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,1,...,11,3,0.00,0,0.000000,7,9,0,4,76.086957


In [ ]:
test13.to_excel('행정동병합테스트22(개발제한행정동제외).xlsx')